In [3]:
import numpy as np
import torch
import re
import pandas as pd
import string

# Transforming text file to csv file

In [4]:
train_path = 'datasets\orig\VLSP2018-SA-Restaurant-train.txt'
dev_path = 'datasets\orig\VLSP2018-SA-Restaurant-dev.txt'
test_path = '\datasets\origVLSP2018-SA-Restaurant-test.txt'

In [5]:
train =open(train_path).read()
valid = open(dev_path).read()
test = open(test_path).read()

In [7]:
def split_aspect(aspects):
    list_sa = [] #aspect and sentiment
    aspects= aspects.split('}, {')
    for i in aspects:
        i = re.sub('{','',i)
        i = re.sub('}','',i)
        i = i.split(', ')
        list_sa.append(i)
    return list_sa


In [55]:
def list_data(file):
    aspect_list = ["AMBIENCE#GENERAL","DRINKS#PRICES", "DRINKS#QUALITY",  "DRINKS#STYLE&OPTIONS", "FOOD#PRICES","FOOD#QUALITY","FOOD#STYLE&OPTIONS","LOCATION#GENERAL", "RESTAURANT#GENERAL", "RESTAURANT#MISCELLANEOUS","RESTAURANT#PRICES","SERVICE#GENERAL"]
    opinions = file.split('\n\n')
    list_sa=[]
    for i in opinions:
        list_temp = []
        if len(i.split("\n")) == 3:
            num, sentence, aspects = i.split("\n")
            aspects = split_aspect(aspects)
            for j in aspects:
                list_temp.append(j[0])
                item = {"sentence": sentence, "aspect":j[0], "sentiment":j[1]}
                list_sa.append(item)
            for aspect in aspect_list:    
                if aspect not in list_temp:
                    item = {"sentence": sentence, "aspect":aspect, "sentiment": "none"}
                    list_sa.append(item)
                else:
                    continue        
        else:
            num, sentence, aspects, space = i.split("\n")
            aspects = split_aspect(aspects)
            for j in aspects:
                item = {"sentence": sentence, "aspect":j[0], "sentiment":j[1]}
                list_sa.append(item)
            for aspect in aspect_list:    
                if aspect not in list_temp:
                    item = {"sentence": sentence, "aspect":aspect, "sentiment": "none"}
                    list_sa.append(item)
                else:
                    continue     
    return list_sa        

In [50]:
def label_encoder(label):
    if label == "positive":
        label_id = 1
    elif label == "negative":
        label_id = 2
    elif label == "neutral":
        label_id = 3
    else:
        label_id = 0
    return label_id    

In [56]:
def transform_data(file):
    data_dict = {"id": [], "sentence": [], "aspect": [], "label_id": [], "label": []}
    list_sa = list_data(file)
    id = 0
    for i in list_sa:
        data_dict["sentence"].append(i['sentence'])
        data_dict["id"].append(str(id))
        data_dict['aspect'].append(i["aspect"])
        data_dict['label_id'].append(label_encoder(i["sentiment"]))
        data_dict['label'].append(i["sentiment"])
        id += 1
        df = pd.DataFrame(data_dict)
    return df

In [57]:
df_train = transform_data(train)
df_val = transform_data(valid)
df_test = transform_data(test)


In [53]:
df_train.head()

,id,sentence,aspect,label_id,label
0,0,"_ Ảnh chụp từ hôm qua, đi chơi với gia đình và...",FOOD#STYLE&OPTIONS,3,neutral
1,1,"_ Ảnh chụp từ hôm qua, đi chơi với gia đình và...",FOOD#QUALITY,3,neutral
2,2,"_Hương vị thơm ngon, ăn cay cay rất thích, nêm...",FOOD#QUALITY,1,positive
3,3,"_Hương vị thơm ngon, ăn cay cay rất thích, nêm...",FOOD#STYLE&OPTIONS,1,positive
4,4,"_Hương vị thơm ngon, ăn cay cay rất thích, nêm...",AMBIENCE#GENERAL,1,positive


In [60]:
df_test.tail()

,id,sentence,aspect,label_id,label
5999,5999,Trà vải ở đây ly khá to và đặc biệt là cho rất...,LOCATION#GENERAL,0,none
6000,6000,Trà vải ở đây ly khá to và đặc biệt là cho rất...,RESTAURANT#GENERAL,0,none
6001,6001,Trà vải ở đây ly khá to và đặc biệt là cho rất...,RESTAURANT#MISCELLANEOUS,0,none
6002,6002,Trà vải ở đây ly khá to và đặc biệt là cho rất...,RESTAURANT#PRICES,0,none
6003,6003,Trà vải ở đây ly khá to và đặc biệt là cho rất...,SERVICE#GENERAL,0,none


In [77]:
train_csv_path = 'train.csv'
df_train.to_csv(train_csv_path, encoding="utf8")
valid_csv_path = 'valid.csv'
df_val.to_csv(valid_csv_path, index = False, encoding="utf8")
test_csv_path = 'test.csv'
df_test.to_csv(test_csv_path,index = False,encoding="utf8")

# working with csv file

In [4]:
train = pd.read_csv('train.csv')
valid = pd.read_csv('valid.csv')
test = pd.read_csv('test.csv')

In [5]:
train.tail()

,id,sentence,aspect,label_id,label
35527,35527,Lạnh trời thế này mà ngồi ăn chả cá lăng xèo x...,LOCATION#GENERAL,0,none
35528,35528,Lạnh trời thế này mà ngồi ăn chả cá lăng xèo x...,RESTAURANT#GENERAL,0,none
35529,35529,Lạnh trời thế này mà ngồi ăn chả cá lăng xèo x...,RESTAURANT#MISCELLANEOUS,0,none
35530,35530,Lạnh trời thế này mà ngồi ăn chả cá lăng xèo x...,RESTAURANT#PRICES,0,none
35531,35531,Lạnh trời thế này mà ngồi ăn chả cá lăng xèo x...,SERVICE#GENERAL,0,none


# Preprocessing

In [5]:
!pip install -q emoji
!pip install -q vncorenlp
!git clone https://github.com/vncorenlp/VnCoreNLP.git

Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 259 (delta 17), reused 14 (delta 2), pack-reused 212
Receiving objects: 100% (259/259), 237.79 MiB | 16.32 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [7]:
def normalize_money(sent):
    return re.sub(r'[0-9]+[.,0-9][k-m-b]', 'giá', sent)

def normalize_hastag(sent):
    return re.sub(r'#+\w+', 'tag', sent)

def normalize_website(sent):
    result = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'website', sent)
    return re.sub(r'\w+(\.(com|vn|me))+((\/+([\.\w\_\-]+)?)+)?', 'website', result)

def nomalize_emoji(sent):
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', sent)

def normalize_elongate(sent):
    patern = r'(.)\1{1,}'
    result = sent
    while(re.search(patern, result) != None):
        repeat_char = re.search(patern, result)
        result = result.replace(repeat_char[0], repeat_char[1])
    return result

def remove_number(sent):
    return re.sub(r'[0-9]+', '', sent)

def normalize_acronyms(sent):
    text = sent
    replace_list = {
        'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
        'okey': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
        '⭐': 'star ', '*': 'star ', '🌟': 'star ', '🎉': u' tích cực ',
        'kg ': u' không ','not': u' không ', u' kg ': u' không ', '"k ': u' không ',' kh ':u' không ','kô':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', '"ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',
        'he he': ' tích cực ','hehe': ' tích cực ','hihi': ' tích cực ', 'haha': ' tích cực ', 'hjhj': ' tích cực ',
        ' lol ': ' tiêu cực ',' cc ': ' tiêu cực ','cute': u' dễ thương ','huhu': ' tiêu cực ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',
        ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' tích cực ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        'dep': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ',
        'iu': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' tích cực ',
        ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback '
    }
    for k, v in replace_list.items():
        text = text.replace(k, v)
    return text


def normalize(sent):
    result = normalize_money(sent)
    result = normalize_hastag(result)
    result = normalize_website(result)
    result = nomalize_emoji(result)
    result = normalize_elongate(result)
    result = normalize_acronyms(result)
    result = remove_number(result)
    result = result.lower()
    return result.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).replace(' '*4, ' ').replace(' '*3, ' ').replace(' '*2, ' ').strip()


In [15]:
def preprocess_data(file_csv):
    for row in range(len(file_csv)):
        sentence = file_csv.loc[row,"sentence"]
        sentence = normalize(sentence)
        file_csv.loc[row,"sentence"] = sentence    

In [16]:
# preprocess_data(train)
preprocess_data(valid)
preprocess_data(test)

In [18]:
test.head()

,id,sentence,aspect,label_id,label
0,0,đây là trong những quán mà mình thích vì vị tr...,RESTAURANT#GENERAL,1,positive
1,1,đây là trong những quán mà mình thích vì vị tr...,DRINKS#QUALITY,1,positive
2,2,đây là trong những quán mà mình thích vì vị tr...,DRINKS#PRICES,3,neutral
3,3,đây là trong những quán mà mình thích vì vị tr...,DRINKS#STYLE&OPTIONS,3,neutral
4,4,đây là trong những quán mà mình thích vì vị tr...,AMBIENCE#GENERAL,0,none


In [19]:
train_prepared_path = 'prepared_train.csv'
train.to_csv(train_prepared_path, index = False, encoding="utf8")
valid_prepared_path = 'prepared_valid.csv'
valid.to_csv(valid_prepared_path, index = False, encoding="utf8")
test_prepared_path = 'prepared_test.csv'
test.to_csv(test_prepared_path,index = False,encoding="utf8")